In [1]:
import sys
sys.path.append("/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR") # dir library

In [32]:
from RADAR.time_series.time_series_utils import TimeSeriesProcessor
from RADAR.time_series.algorithms import tsfedl
from RADAR.time_series.algorithms import transformers
from RADAR.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
from RADAR.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from RADAR.visualization_module import DataVisualization,DataVisualizationScoresTS
import numpy as np

### Experiments Time Series: Example with AI4I 2020 Predictive Maintenance Dataset

In [34]:
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding
print(y)


Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [35]:
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(8000, 5) (2000, 5) (8000,) (2000,)


In [36]:
wsize = 24
step = 1
n_pred = 1
features = 5


processor = TimeSeriesProcessor(window_size= wsize, step_size=1, future_prediction=False, n_pred=n_pred)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 5)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 5)
y_test shape: (1977, 24)


In [37]:
X_train_windows = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_windows = torch.tensor(y_train_windows, dtype=torch.float32)
y_train_windows = y_train_windows.unsqueeze(-1)  # to (7977, 24, 1)



#### Define model Ohshulih

In [15]:
from TSFEDL.models_pytorch import OhShuLih_Forecaster
top_module = OhShuLih_Forecaster(out_features= features,n_pred=n_pred)
top_module.in_features

20

In [18]:

kwargs = {"algorithm_": "ohshulih", "loss": torch.nn.MSELoss(),"top_module": top_module, "max_epochs": 100, "in_features":wsize}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)



{'label_parser': None, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 24, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}


In [19]:
model1.fit(X_train_windows,y_train_windows)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name         | Type                | Params | Mode 
-------------------------------------------------------------
0 | loss         | MSELoss             | 0      | train
1 | classifier   | OhShuLih_Forecaster | 105    | train
2 | convolutions | Sequential          | 1.8 K  | train
3 | lstm         | LSTM                | 2.2 K  | train
-------------------------------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.017     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode
/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10

Training: |          | 0/? [00:00<?, ?it/s]

/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 1])) that is different to the input size (torch.Size([16, 1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 1])) that is different to the input size (torch.Size([9, 1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=100` reached.


In [20]:
scores_pred = model1.decision_function(X_test_windows)
print("Scores",scores_pred)


Scores [0.7812354  0.84763324 0.83241874 ... 1.0447831  1.5137095  1.5000453 ]


In [21]:
y_pred = model1.predict(X_test_windows)
y_pred.shape

(1977,)

In [24]:
y_pred =np.array(y_pred).ravel()
y_true = (y_test_windows.sum(axis=1) > 0).astype(int)  # shape (1977,)

In [25]:
y_pred == y_true

array([False, False, False, ...,  True,  True, False])

In [27]:

visualizer = DataVisualizationScoresTS(scores_pred)
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()

In [30]:
result = model1.evaluate(X_test_windows)
result

{'scores': array([0.7812354 , 0.84763324, 0.83241874, ..., 1.0447831 , 1.5137095 ,
        1.5000453 ], dtype=float32),
 'labels_preds': array([0, 0, 0, ..., 0, 1, 0]),
 'preds': array([0, 0, 0, ..., 0, 1, 0])}

#### Define model Informer

In [40]:
batch_size = 64
seq_len = wsize
input_dim = features  # tanto encoder como decoder
d_model = 64
train_epochs = 100

# Simulamos series de entrada
X = torch.randn(batch_size, seq_len, input_dim)

# Instanciamos el modelo Transformer
kwargs ={
    "algorithm_": "transformer",
    "label_parser": None,
    "size_enc_in":input_dim,
    "size_dec_in": input_dim,
    "ulayers_feedfwd": 128,
    "seq_len":seq_len,
    "d_qk":64,
    "d_v":64,
    "d_model":d_model,
    "n_layers": 2,
    "n_heads": 8,
   "dropout_rate": 0.1,
   "attns_outs":False, 
    "train_epochs": train_epochs,
    "batch_size": batch_size,
    "lr": 1e-3
    }
    

model = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': None, 'train_epochs': 100, 'batch_size': 64, 'lr': 0.001} 
Model Params: {'algorithm_': 'transformer', 'size_enc_in': 5, 'size_dec_in': 5, 'ulayers_feedfwd': 128, 'seq_len': 24, 'd_qk': 64, 'd_v': 64, 'd_model': 64, 'n_layers': 2, 'n_heads': 8, 'dropout_rate': 0.1, 'attns_outs': False}


In [41]:

model.fit(X_train_windows)


/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([64, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([41, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1/100, Loss: 1.042106
Epoch 2/100, Loss: 1.013116
Epoch 3/100, Loss: 1.009673
Epoch 4/100, Loss: 1.006538
Epoch 5/100, Loss: 1.005305
Epoch 6/100, Loss: 1.004690
Epoch 7/100, Loss: 1.003838
Epoch 8/100, Loss: 1.003383
Epoch 9/100, Loss: 1.002820
Epoch 10/100, Loss: 1.002707
Epoch 11/100, Loss: 1.002432
Epoch 12/100, Loss: 1.002279
Epoch 13/100, Loss: 1.002251
Epoch 14/100, Loss: 1.001745
Epoch 15/100, Loss: 1.002469
Epoch 16/100, Loss: 1.002232
Epoch 17/100, Loss: 1.001567
Epoch 18/100, Loss: 1.001065
Epoch 19/100, Loss: 1.001482
Epoch 20/100, Loss: 1.001810
Epoch 21/100, Loss: 1.001388
Epoch 22/100, Loss: 1.000874
Epoch 23/100, Loss: 1.000961
Epoch 24/100, Loss: 1.000629
Epoch 25/100, Loss: 1.001114
Epoch 26/100, Loss: 1.000999
Epoch 27/100, Loss: 1.000695
Epoch 28/100, Loss: 1.000543
Epoch 29/100, Loss: 1.000313
Epoch 30/100, Loss: 1.000672
Epoch 31/100, Loss: 1.000324
Epoch 32/100, Loss: 1.000104
Epoch 33/100, Loss: 1.000291
Epoch 34/100, Loss: 1.000037
Epoch 35/100, Loss: 0.9

In [42]:
scores = model.decision_function(X_train_windows)
labels = model.predict(X_train_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", labels.shape)


Anomaly scores: torch.Size([7977])
Predicted labels: torch.Size([191448, 5])


In [45]:
model.evaluate(X_test_windows,y_test_windows)

Accuracy: 96.267%
F1 Score: 0.138
Recall: 0.101
Precision: 0.217


{'scores': array([ 0.47580156,  1.0704737 ,  0.54898894, ...,  1.6335933 ,
        12.154701  ,  1.577977  ], dtype=float32),
 'labels_preds': array([0, 0, 0, ..., 0, 1, 0]),
 'labels_true': tensor([0, 1, 0,  ..., 0, 1, 0])}

In [50]:

visualizer = DataVisualizationScoresTS(scores[:3000])
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()

### Experiments Time Series: Example with Metro Interstate Traffic Volume

In [67]:
X,y = load_time_series('metro_interstate_traffic_volume')
labels = y["traffic_volume"]
X = X.drop(["date_time", "holiday","weather_main","weather_description"], axis=1)

Metadata: {'uci_id': 492, 'name': 'Metro Interstate Traffic Volume', 'repository_url': 'https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume', 'data_url': 'https://archive.ics.uci.edu/static/public/492/data.csv', 'abstract': 'Hourly Minneapolis-St Paul, MN traffic volume for westbound I-94. Includes weather and holiday features from 2012-2018.', 'area': 'Other', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Sequential', 'Time-Series'], 'num_instances': 48204, 'num_features': 8, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['traffic_volume'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2019, 'last_updated': 'Fri Mar 15 2024', 'dataset_doi': '10.24432/C5X60B', 'creators': ['John Hogue'], 'intro_paper': None, 'additional_info': {'summary': 'Hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN. Ho

In [68]:
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(38563, 4) (9641, 4) (38563,) (9641,)


In [74]:
wsize = 24
step = 1
n_pred = 1
features= 4
processor = TimeSeriesProcessor(window_size= wsize, step_size=1, future_prediction=False, n_pred=n_pred)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)
#test

X_train shape: (38540, 24, 4)
y_train shape: (38540, 24)
X_test shape: (9618, 24, 4)
y_test shape: (9618, 24)


In [75]:
X_train_windows = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_windows = torch.tensor(y_train_windows, dtype=torch.float32)
y_train_windows = y_train_windows.unsqueeze(-1)  # to (7977, 24, 1)


In [77]:
from TSFEDL.models_pytorch import OhShuLih_Forecaster
top_module = OhShuLih_Forecaster(out_features= features,n_pred=n_pred)
top_module.in_features

20

In [83]:

kwargs = {"algorithm_": "ohshulih", "loss": torch.nn.MSELoss(),"top_module": top_module, "max_epochs": 100, "in_features":wsize,"batch_size":32}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)

{'label_parser': None, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 24, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}


In [84]:
model1.fit(X_train_windows,X_train_windows)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs

  | Name         | Type                | Params | Mode 
-------------------------------------------------------------
0 | loss         | MSELoss             | 0      | train
1 | classifier   | OhShuLih_Forecaster | 84     | train
2 | convolutions | Sequential          | 1.8 K  | train
3 | lstm         | LSTM                | 2.2 K  | train
-------------------------------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([32, 24, 4])) that is different to the input size (torch.Size([32, 1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Detected KeyboardInterrupt, attempting graceful shutdown ...


TSFEDLerror fit():  name 'exit' is not defined
For further reference please see: https://s-tsfe-dl.readthedocs.io/en/latest/index.html


NameError: name 'exit' is not defined

In [ ]:
scores_pred = model1.decision_function(X_test_windows)
print("Scores",scores_pred)

In [ ]:
y_pred = model1.predict(X_test_windows)
y_pred.shape

In [ ]:
y_pred =np.array(y_pred).ravel()
y_true = (y_test_windows.sum(axis=1) > 0).astype(int)  # shape (1977,)

In [ ]:

visualizer = DataVisualizationScoresTS(scores_pred)
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()